In [ ]:
%reload_ext autoreload
%aimport -pandas, -numpy
%autoreload 2
import pandas as pd; pd.options.mode.chained_assignment = None # default warn => SettingWithCopyWarning
import numpy as np
from datetime import datetime
from pprint import pprint as pp
import warnings; warnings.simplefilter(action='ignore', category=UserWarning)
from IPython.display import HTML, display
from ipywidgets import AppLayout, Button, Layout, VBox, HBox, Label, HTML, interact, IntSlider
import ipywidgets as widgets
import arrow
from tqdm.auto import tqdm

#import dmyplant2
from dmyplant2 import (
    cred, MyPlant, Engine, 
    msgFSM, filterFSM, FSM_splot, FSM_add_Notations, FSM_add_Alarms, FSM_add_Warnings, FSMPlot_Start, 
    bokeh_show, dbokeh_chart, add_dbokeh_vlines, get_cycle_data2, 
    disp_result, alarms_pareto, warnings_pareto, states_lines,
    detect_edge_right, detect_edge_left, cvset, load_data, get_cycle_data, get_cycle_data2, figures)
    
cred()
mp = MyPlant(0)
Engine._list_cached_validations();
#mp._fetch_installed_base(); # refresh local installed fleet database

In [ ]:
def sfun(x):
    return all([ ("EWB" in str(x['IB Site Name'])),  (x['OperationalCondition'] != 'Decommissioned') ])
    #return all([ ("Pforzheim" in str(x['IB Site Name'])),  (x['OperationalCondition'] != 'Decommissioned') ])
    #return all([ ("BMW Landshut 4.10" in str(x['IB Site Name'])),  (x['OperationalCondition'] != 'Decommissioned') ])
fleet = mp.search_installed_fleet(sfun).drop('index', axis=1)
fleet = fleet.sort_values(by = "Engine ID",ascending=True).reset_index(drop='index')
fleet.T

In [ ]:
motor = fleet.iloc[2]
modes = ['???','OFF','MANUAL','AUTO']
success = [True,False]
#success = [True]
e=Engine.from_fleet(mp,motor)
#pp_from=e['Commissioning Date']
pp_from='2021-01-01'
pp_to='2022-03-30'
#pp_to=datetime.now()
fsm = msgFSM(e, p_from=pp_from, p_to=pp_to)
#TODO:
# - move successtime to run1
motor['IB Site Name'] + ' ' + motor['Engine ID'], modes, success

### den queue algorithmus entwerfen

In [ ]:
def consume_testmessage_queue(message_queue):
    return []

In [ ]:
def fill_testmessage_queue(mq, d, dt):
    z = next(d)
    zr = z
    while zr < z+dt:
        mq.append(zr)
        try:
            zr = next(d)
        except StopIteration:
            mq.pop()
            break
    mq.append(zr)
    return mq

In [ ]:
def merge_extra_testmessages(mq, em):
    emc = em.copy()
    for e in emc:
        if e < max(mq):
            mq.append(e)
            em.remove(e)
    mq = sorted(mq)
    return mq, em

In [ ]:
d = list(range(0,56,2)).__iter__()
mq = []
em = [3,7, 37,53,21]

while True:
    try:
        mq = fill_testmessage_queue(mq, d, 20)
        print('after fillmessage:    ',mq)
        print('extramessages:        ',em)
        mq, em = merge_extra_testmessages(mq, em)
        print('merge extramessages:  ',mq)
        print('extramessages:        ',em)
        mq = consume_testmessage_queue(mq)
        print('after consume mgs:    ',mq)
        print()
    except StopIteration:
        print(f"Ende.")
        break

### mit echten messages:

In [ ]:
import pandas as pd

In [ ]:
def fill_message_queue(mq, d, dt):
    """fills a message queue with dt time lenght

    Args:
        mq (list): message que
        d (iterator): message iterator variable
        dt (int): delta t in min

    Returns:
        _type_: _description_
    """
    i, m = next(d)
    t0 = m['timestamp'] 
    #print('von: ',t0, pd.to_datetime(t0 * 1000000))
    dtns = dt*60*1000
    #print('bis: ',t0 + dtns, pd.to_datetime((t0 + dtns) * 1000000))
    mit = m
    while mit['timestamp'] < t0 + dtns:
        mq.append(mit)
        try:
            i, mit = next(d)
        except StopIteration:
            mq.pop()
            break
    mq.append(mit)
    return mq

In [ ]:
def merge_extra_messages(mq, em):
    emc = em.copy()
    max_timestamp = max([ts['timestamp'] for ts in mq])
    for m in emc:
        if m['timestamp'] < max_timestamp:
            mq.append(m)
            em.remove(m)
    mq.sort(key=lambda x: x['timestamp'], reverse=False)
    return mq, em

In [ ]:
def consume_message_queue(message_queue):
    # run the finite state machine
    return []

In [ ]:
def debug_msg(msgque, max=10):
    return
    print(len(msgque))
    for msg in msgque[:max]:
        print(f"{msg['timestamp']} {pd.to_datetime(msg['timestamp'] * 1000000)} {msg['name']} {msg['message']}")
    print()

In [ ]:
from extramessages import extra_messages
em = extra_messages.copy()
it = fsm._messages.iterrows()
mq = []
#.strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
from tqdm.auto import tqdm

pbar = tqdm(total=len(fsm._messages), ncols=80, mininterval=1, unit=' messages', desc="FSM")
#tqdm(self._messages.iterrows(), total=self._messages.shape[0], ncols=80, mininterval=1, unit=' messages', desc="FSM"):

while True:
    try:
        mq = fill_message_queue(mq, it, 5*60)
        debug_msg(mq)
        debug_msg(em)
        pbar.update(len(mq))

        mq, em = merge_extra_messages(mq, em)
        debug_msg(mq)
        debug_msg(em)

        mq = consume_message_queue(mq)
        debug_msg(mq)
    except StopIteration:
        print(f"{len(fsm._messages)} messages scanned.")
        break

pbar.close()

### first playground for message queue

In [ ]:

m = next(it)
t0 = m['timestamp']
message_queue.append(m)

while m['timestamp'] < t0 + 1*60*60*1000: # 1h
    t = next(it)
    message_queue.append(t)

message_queue = consume_message_queue(message_queue)

In [ ]:
t0 = message_queue[0][1]['timestamp']
for m in message_queue:
    print(f"{m[0]:04} {m[1]['timestamp']:10d} {(m[1]['timestamp'] - t0):10d} sec {m[1]['name']} {m[1]['message']}")

In [ ]:
itt = fsm._messages.itertuples()

In [ ]:
t = next(itt)
t.timestamp, t.name, t.message

### Tests for tdqm usage

In [ ]:
from tqdm.auto import tqdm
import time
_messages = [n for n in range(10)]
_messages

In [ ]:
pbar = tqdm(total=len(_messages), ncols=80, mininterval=1, unit=' messages', desc="FSM")
#pbar = tqdm(total=len(_messages), ncols=80, unit=' messages', desc="FSM")
count = 0
for i, m in enumerate(_messages):
        #print(m)
        count = count + 1
        time.sleep(1)
        pbar.update()
pbar.close()   

In [ ]:
with tqdm(total=len(_messages),ncols=80, unit=' messages', desc="FSM") as pbar:
    count = 0
    for i, m in enumerate(_messages):
            count = count + 1
            time.sleep(1)
            pbar.update(1)            

In [ ]:
from tqdm.auto import tqdm
count = 0
for m in tqdm(range(10), ncols=80, unit=' messages', desc="FSM"):
        #print(m)
        count = count + 1
        time.sleep(1)

In [ ]:
from tqdm.auto import tqdm, trange
count = 0
for m in trange(100, ncols=80, unit=' messages', desc="FSM"):
        count = count + 1
        time.sleep(0.05)

In [ ]:
var = 'Exhaust_TempCyl*'
var.__contains__('*')
var.endswith('*')

In [ ]:
import dmyplant2
from dmyplant2 import MyPlant
dmyplant2.cred()
mp = dmyplant2.MyPlant(0)
e=dmyplant2.Engine.from_sn(mp,1486174)
#e=dmyplant2.Engine.from_sn(mp,1171430)

In [ ]:
e.dataItemsCyl('Exhaust_TempCyl*')

In [ ]:
v = [var.replace('*',f"{i+1:02d}") for i in range(e.Cylinders)]
v

In [ ]:
e.get_dataItems(v)

In [ ]:
import arrow
e.assess_dataItems(v, arrow.get('2022-01-14 16:23:06'))